In [ ]:
# Simple random 0/1 generator
# Just Hadamard and measure

from projectq import MainEngine
from projectq.ops import H, Measure, Ry

def myfunc(eng):
    qubit = eng.allocate_qubit()
    
    H | qubit
    # Ry(-3.1416/2) | qubit
    Measure | qubit
    eng.flush()
    return(int(qubit))

eng = MainEngine()
j=0
for i in range(1,100):
    j+=myfunc(eng)
    
print("#1s: {}".format(int(j)))

In [ ]:
# Draw a circuit for Bell-pair preparation

from projectq import MainEngine 
from projectq.ops import H, CNOT
from projectq.backends import CircuitDrawer

def create_bell_pair(eng):
    b1 = eng.allocate_qubit()
    b2 = eng.allocate_qubit()
    
    H | b1
    CNOT | (b1,b2)
    
    return b1, b2

# create the main compiler engine 
drawing_engine = CircuitDrawer()
eng = MainEngine(drawing_engine)

create_bell_pair(eng)

eng.flush()
print(drawing_engine.get_latex())


In [ ]:
# Implementing the teleportation protocol

from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import All, H, CNOT, Measure, Rz, X, Z
from projectq.meta import Dagger, Control 

def create_bell_pair(eng):
    b1 = eng.allocate_qubit()
    b2 = eng.allocate_qubit()
    
    H | b1
    CNOT | (b1, b2)
    
    return b1, b2

def run_teleport(eng, state_preparation_map, verbose=False):
    # first make a Bell pair
    b1, b2 = create_bell_pair(eng)
    
    # let Alice prepare her state
    psi = eng.allocate_qubit()
    if verbose:
        print("Alice is preparing her state from scratch, i.e. |0>...")
    state_preparation_map(eng, psi)
       
    
    # entangle Alice's state with her Bell qubit b1
    CNOT | (psi, b1)
    if verbose:
        print("Alice has entangled her state with her side of the Bell.")
        
    # rotate to Hadamard basis, measure psi and b1
    H | psi
    Measure | psi
    Measure | b1
    msg_to_bob = [int(psi),int(b1)]
    if verbose:
        print("Alice is sending msg {} to Bob".format(msg_to_bob))
        
    # Bob makes his corrections
    with Control(eng, b1):
        X | b2
    with Control(eng, psi):
        Z | b2
        
    # cheat and print the state
    print(eng.backend.cheat())
        
    # Bob tries to uncompute the b2 state using the psi creator
    if verbose:
        print("Bob is trying to uncompute the state he received...")
    with Dagger(eng):
        state_preparation_map(eng, b2)
    
    # Check if the uncompute was successful
    # The simulator only allows deleting qubits in computational basis states
    del b2
    eng.flush()
    
    if verbose:
        print("Bob successfully arrived at state |0> after uncomputing")
        
    
if __name__ == "__main__":
    # Create main compiler engine with a simulator backend
    # eng = MainEngine()
    
    # Create main engine with a circuit drawer backend 
    drawing_engine = CircuitDrawer()
    eng = MainEngine()
    
    # define state preparation map for the state to be teleported
    # it takes |0> as input
    # Alice uses it to prepare a state and teleport it to bob
    # Bob uses it to uncompute the received state
    
    def create_state(eng, qb):
        H | qb
        Rz(1.21) | qb
        
    # run the teleportation protocol and let Bob uncompute his qubit
    run_teleport(eng, create_state, verbose=False)
    
    
    # print(drawing_engine.get_latex())
        

        

In [ ]:
def projector(dim, index):
        size = pow(2, dim)
        if (index >= size):
            print("index out of bounds, throwing exception...")
            return None
        a = np.zeros(size)
        a[index]=1
        matrix = np.outer(a, a)
        return matrix
    
def lcu_controlled_unitary(list_of_U):
    size = len(list_of_U) 
    system_size = len(list_of_U[0].matrix[:,1])              # hack for square matrices
    control_dim = math.ceil(math.log(size, 2))
    mat_dim = 2**control_dim * system_size
    
    target = np.zeros((mat_dim,mat_dim))
    for i in range (0,size):
        proj_i = projector(control_dim, i)
        sysmat = np.asmatrix(list_of_U[i].matrix)
        temp = np.kron(proj_i, sysmat)
        target = np.add(target, temp)
      
    U = Unitary(target)
    return U


# Testing the print(eng.backend.cheat()) function
# understand and document its behaviour

from projectq import MainEngine
from projectq.ops import H, Measure, Ry, CNOT, X, Z, All
from projectq.meta import Dagger, Control

import math

eng = MainEngine()

n = 3
psi = eng.allocate_qureg(n)
eng.flush()
print(eng.backend.cheat())
X | psi[2]
eng.flush()
print(eng.backend.cheat())
All(Measure) | psi

A = [X, Z]

All(A[1]) | psi
eng.flush()
print(eng.backend.cheat())
All(Measure) | psi

In [ ]:
def fpoaa_string(depth):
    if(depth==1):
        return "WRMRW"
    else:
        V = fpoaa_string(depth-1)
        U = dagger(V)
        return V + 'R' + ''.join(U) + 'R' + V
    
def dagger(source):
    string = list(source)
    m = len(string)
    temp = [None] * m
    
    for i in range(0, m):
        if (string[m-1-i]=='W'):
             temp[i] = 'M'
        elif (string[m-1-i]=='M'):
             temp[i] = 'W'
        elif (string[m-1-i]=='R'):
             temp[i] = 'S'
        elif (string[m-1-i]=='S'):
            temp[i] = 'R'
    return temp

n = 2
string = fpoaa_string(n)
print(string)



In [ ]:

"Hello"[-1:3]

In [2]:
from projectq.meta import Compute, Uncompute, Control
from projectq import MainEngine
from projectq.ops import H, Measure, Ry, CNOT, X, Z, All
import numpy as np
import math


def lcu_control_state_prep(eng, coefts, reg, dim):
    m = len(coefts)
    # dim = math.ceil(math.log(m,2)) 
    
    size = pow(2, dim)
    
    weight = np.sum(coefts)
    probs = np.zeros(size)
    probs[0:m] = coefts
    probs = (1.0/weight) * probs 
    # print("prob vec: {}".format(probs))
    
    # compute the rotation angles required for preparing
    # the LCU control state, apply conditional rotations
    # following the method of Grover & Rudolph (2002)
    
    # we need to perform bifurcations over dim rounds
    
    for i in range(1,dim+1):
        block_size = 2**(dim-i+1)
        # print("block_size: {}".format(block_size))
               
        # in each round, need to bifurcate 2^(round - 1) blocks
        num_blocks = 2**(i-1)
#        print("num_blocks: {}".format(num_blocks))
        target = np.zeros((2**i, 2**i))
        for j in range(0, num_blocks):
            # break loop if we've already crossed the last non-zero coefficient
            if (j*block_size > m-1):
                print("Broke from j={} because we crossed the last element\n".format(int(j)))
                break
            
            start = j*block_size
            end = start + block_size
#            print("start: {}, end: {}".format(int(start),int(end)))
            
            vec_j = probs[start : end]
#            print("block to be bifurcated: {}".format(vec_j))
            # break loop if singleton
            if (len(vec_j)<=1):
                print("Broke from j={} because we encountered a singleton\n".format(int(j)))
                break
            
            left_cond_prob_j = bifurcate(vec_j)
            # print("left cond prob of vec_j: {}".format(left_cond_prob_j))
            
            # perform a rotation with angle corresponding to sqrt of this prob
            f_j = math.sqrt(left_cond_prob_j)
            ang_j = math.acos(f_j)
            
            temp = np.binary_repr(j)
            temp = temp.zfill(i-1)        # pad with zeros for fixed length bin
            with Compute(eng):
                for j in range(0,i-1):
                    if (int(temp[j])==0):
                        X | reg[j]
            
            with Control(eng, reg[0:i-1]):
                Ry(2*ang_j) | reg[i-1]
            Uncompute(eng)
        
#         eng.flush()
#         print("at iteration {}, we have the state \n".format(int(i)))
#         print(eng.backend.cheat())

# given a vector, bifurcate it into left and right 
# halves, and return the conditional
# probability of the left half
# will always expect len(vec) a power of 2
def bifurcate(vec):
    m = len(vec)
    tot = np.sum(vec)
    if (tot==0):
        print("some segment has probability 0")
        return 0
    
    left = 0.0
    for i in range (0, m//2):          # // = int division op
        left += vec[i]
        
    return left/tot


eng=MainEngine()

coefts = [2, 1, 2, 1, 2]
m = len(coefts)
sys_dim=0
ctrl_dim=math.ceil(math.log(m,2))
maxout = pow(2, ctrl_dim)
dim=sys_dim + ctrl_dim
print("num qubit = {} \n".format(int(dim)))
reg = eng.allocate_qureg(dim)

eng.flush()
print("state of allocated qureg[{}] \n".format(int(dim)))
print(eng.backend.cheat())


with Compute(eng):
    lcu_control_state_prep(eng, coefts, reg, ctrl_dim)

eng.flush()
print("After lcu_state_prep, we have the state \n")
print(eng.backend.cheat())

for i in range(0, maxout):
    bin_i = np.binary_repr(i)
    bin_i = bin_i.zfill(ctrl_dim)        # padding with zeros to get fixed length bin
    temp = eng.backend.get_amplitude(bin_i, reg)
    print("amplitude of string {} (i.e of {}) is {}".format(bin_i, int(i), temp))

Uncompute(eng)
eng.flush()
print("After lcu_state_un_prep, we have the state \n")
print(eng.backend.cheat())

All(Measure) | reg

num qubit = 3 

state of allocated qureg[3] 

({0: 0, 1: 1, 2: 2}, [(1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j])
Broke from j=3 because we crossed the last element

After lcu_state_prep, we have the state 

({0: 0, 1: 1, 2: 2}, [(0.49999999999987627+0j), (0.5000000000001742+0j), (0.499999999999928+0j), 0j, (0.35355339059327073+0j), 0j, (0.3535533905933073+0j), 0j])
amplitude of string 000 (i.e of 0) is (0.49999999999987627+0j)
amplitude of string 001 (i.e of 1) is (0.35355339059327073+0j)
amplitude of string 010 (i.e of 2) is (0.499999999999928+0j)
amplitude of string 011 (i.e of 3) is (0.3535533905933073+0j)
amplitude of string 100 (i.e of 4) is (0.5000000000001742+0j)
amplitude of string 101 (i.e of 5) is 0j
amplitude of string 110 (i.e of 6) is 0j
amplitude of string 111 (i.e of 7) is 0j
After lcu_state_un_prep, we have the state 

({0: 0, 1: 1, 2: 2}, [(1+0j), (-8.62088178621434e-14+0j), (-6.461154626247702e-14+0j), (3.730349362742383e-14+0j), (-6.45876478867389e-14+0j), (3.728969589375822

TypeError: unsupported operand type(s) for *: 'int' and 'XGate'

In [1]:
A = ['x','YZ']
print(str(A[1]))
num = [1, 2, 3, 4]
print(num)
num[2] += 2
print(num[2])

YZ
[1, 2, 3, 4]
5
